In [ ]:
# First approach to Smart Charging Phase 1 using PuLP.
# Modeled as a PuLP optimisation blending problem
# Started 20 Aug 2020
# Author: Sofia Taylor, Flexible Power Systems

import numpy as np
import pandas as pd
import datetime as dt
from pulp import *
import pickle
import global_variables as gv
import functions as f
import output_functions as of
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import random
import matplotlib.dates as mdates
import matplotlib.font_manager as font_manager
import lin_prog_functions as lpf


In [ ]:
#grid_file_path = 'Outputs/Logs/grid_variables{}.csv'.format(run)
journeys = pickle.load(open('Outputs/journeys_range','rb'))
eprice = pickle.load(open('Outputs/price_data','rb'))
empty_profile = pickle.load(open('Outputs/empty_profile','rb'))

In [ ]:
run = 74
charger = 22
capacity = 30
ca='opt'
notes = 'Introduced a magic charging step when completely unfeasible'
site_capacity = {
    'opt': capacity,  
    'BAU': 10000,
    'BAU2': capacity
}
script_strt = time.process_time()
print('Run:',run,'/ Charger:',charger,'/ Capacity:',capacity)
os.makedirs('Outputs/Logs/run{}'.format(run))

profile_out, dates, bad_days, lpprob = optimise_range2(
    empty_profile, 
    charger, 
    site_capacity)

range_profile, site_profile, days_summary, global_summary = of.summary_outputs(
profile_out, 
    journeys, 
    dates)

os.makedirs('Outputs/Logs/run{}/daily'.format(run))
for date in dates:
    day = dt.datetime.combine(date, dt.datetime.min.time())
    day_profile = of.create_daily_summary(site_profile, day)
    fig_summary = of.summary_plot(day_profile)
    fig_summary.savefig(
        'Outputs/Logs/run{}/daily/fig{}.jpg'.format(run,date))
    plt.close(fig_summary)

with open('Outputs/Logs/run{}/variables{}.csv'.format(run,run),'a') as fi:
    fi.write(notes)
    fi.write('\n' + str(run)+'\n'+str(charger) + '\n' + str(capacity) +'\n')
    fi.write(global_summary.to_string())
    fi.write(bad_days)

range_fig = of.daily_summary_plot(days_summary.fillna(0))
range_fig.savefig(
            'Outputs/Logs/run{}/fig_range{}.svg'.format(run,run),
            bbox_inches = "tight")
range_fig.show()

In [ ]:
# Optimise over a whole range
def optimise_range2(empty_profile, charger, capacity):
    dates = np.unique(empty_profile.index.get_level_values(0).date)
    #dates = np.delete(dates,-1)
    all_days_profile = []
    dates_status = ''
    bad_days = 'Bad days:\n'
    status = 0
    initial_rel_charge = pd.Series(
        data = [0,0,0,0,0],
        index = empty_profile.index.get_level_values(1).unique()
    )
    rel_charge = dict.fromkeys(gv.CATS,initial_rel_charge)

    for date in dates:
        day_status = 0
        start = time.process_time()
        day = dt.datetime.combine(date, dt.datetime.min.time())
        day_profile = f.create_daily_schedule(empty_profile, day)
        if len(day_profile)==0:
            bad_days += '\nEmpty day:'
            bad_days += str(date)
            pass
        else:
            output_df = {}
            PuLP_prob = {}
            day_profile_out = day_profile.copy()
            for ca in gv.CATS:
                output_df[ca], PuLP_prob[ca], rel_charge[ca] = linear_optimiser_V3(
                    day_profile,
                    ca,
                    charger,
                    capacity,
                    rel_charge[ca]
                    )
                day_profile_out = day_profile_out.merge(
                    output_df[ca],
                    how='left',
                    left_index=True,
                    right_index=True,
                    )
                day_profile_out.fillna(0,inplace=True)
                day_status += PuLP_prob[ca].status
            
            print(
                date,
            #     '\nTime:', time.process_time() - start,
                'Status:',day_status, 
                ':', PuLP_prob['opt'].status, 
                PuLP_prob['BAU'].status,
                PuLP_prob['BAU2'].status)
            #     '\nCost:', value(PuLP_prob['opt'].objective))
            all_days_profile.append(day_profile_out)
            if day_status <3:
                bad_days += '\nNon-Optimal: '
                bad_days += str(date)
                for ca in gv.CATS:
                    bad_days += '_'
                    bad_days += str(PuLP_prob[ca].status)
            dates_status += str([date,day_status])
            dates_status += '\n'

    profile_out = pd.concat(all_days_profile)
    return profile_out, dates, bad_days, PuLP_prob

# Passes incomplete SoC to next day
def linear_optimiser_V3(profile,ca,charger,capacity,rel_charge):
    profile_av = profile[profile['Available'] == 1]
    price_col = gv.CAT_COLS['PRICE'][ca]
    output_col = gv.CAT_COLS['OUTPUT'][ca]
    vehicles = profile.index.get_level_values(1).unique()
    time_periods = profile_av.index.get_level_values(0).unique()
    # Define output variable
    outputs = LpVariable.dicts(
        "output",
        ((period, vehicle) for period, vehicle in profile_av.index),
        lowBound = 0,
        upBound = charger * gv.TIME_FRACT,
        cat = "Continuous"
        )

    # Create the 'prob' variable to contain the problem data
    prob = LpProblem("Multiple_route_scheduling",LpMinimize)

    # Add costs to objective function
    prob += lpSum(
        [profile_av.loc[(period, vehicle), price_col] * outputs[period, vehicle] 
        for period, vehicle in profile_av.index]
        ), "Total Charging costs"

    # Final SOC constraint
    for vehicle in vehicles:
        vehicle_prof = profile_av.loc[(slice(None),vehicle),'Battery_Use']
        prob += lpSum(
            [outputs[period,vehicle] * gv.CHARGER_EFF for period, vehicle in vehicle_prof.index]
        ) == - (
            profile.loc[(slice(None),vehicle),'Battery_Use'].sum()
            + rel_charge[vehicle]
        )

    # Intermediate SOC constraints
    for vehicle in vehicles:
        vehicle_prof = profile_av.loc[(slice(None),vehicle),'Battery_Use']
        for period in vehicle_prof.index.get_level_values(0):
            cumul_use = profile.loc[(slice(period),vehicle),'Battery_Use'].sum()
            cumul_profile = profile_av.loc[(slice(period),vehicle),'Battery_Use']
            prob += lpSum( # Doesn't go over 100% SOC
                [outputs[period, vehicle] * gv.CHARGER_EFF for period, vehicle in cumul_profile.index]
            ) + cumul_use + rel_charge[vehicle]<= 0
            prob += lpSum( # Doesn't go below 0% SOC
                [outputs[period, vehicle] * gv.CHARGER_EFF for period, vehicle in cumul_profile.index]
            ) + cumul_use + rel_charge[vehicle]>= -gv.BATTERY_CAPACITY + charger * gv.TIME_FRACT

    # Max capacity constraint
    n = len(time_periods.unique())
    for period in time_periods:
        time_routes = list(profile_av.loc[period].index)
        prob += lpSum(
            [outputs[period, route] for route in time_routes]) <= capacity[ca]* gv.TIME_FRACT

    # Solve and print to the screen
    prob.solve()
    print(ca, "status:", LpStatus[prob.status])
    # If unfeasible, tries to 
    if  prob.status == -1:
        df = charge_incomplete(profile,ca,charger,capacity,rel_charge)
    else:
        # Get output variables
        charge_output = []
        for period, route in outputs:
            if prob.status == 1:
                x =  outputs[(period, route)].varValue
            else:
                x = 0
            var_output = {
                'from': period,
                'Vehicle_ID': route,
                output_col: x
            }
            charge_output.append(var_output)

        df = pd.DataFrame.from_records(charge_output).sort_values(['from','Vehicle_ID'])
        df.set_index(['from', 'Vehicle_ID'], inplace=True)
        print('Cost:', value(prob.objective))
    # Generate a final SoC array
    #final_soc = initial_soc
    final_soc = rel_charge + (
        df.groupby('Vehicle_ID').sum()[output_col]*gv.CHARGER_EFF
        + profile.groupby('Vehicle_ID').sum()['Battery_Use'] )
    return df, prob, final_soc

# Incomplete charging when 100% unfeasible
def charge_incomplete(profile,ca,charger,capacity,rel_charge):
    profile_av = profile[profile['Available'] == 1]
    price_col = gv.CAT_COLS['PRICE'][ca]
    output_col = gv.CAT_COLS['OUTPUT'][ca]
    vehicles = profile.index.get_level_values(1).unique()
    time_periods = profile_av.index.get_level_values(0).unique()
    # Define output variable
    outputs = LpVariable.dicts(
        "output",
        ((period, vehicle) for period, vehicle in profile_av.index),
        lowBound = 0,
        upBound = charger * gv.TIME_FRACT,
        cat = "Continuous"
        )

    # Create the 'prob' variable to contain the problem data
    prob = LpProblem("Multiple_route_scheduling",LpMaximize)

    # Add costs to objective function
    prob += lpSum(
        [outputs[period, vehicle] for period, vehicle in profile_av.index]
        ), "Total Charging"

    # Intermediate SOC constraints
    for vehicle in vehicles:
        vehicle_prof = profile_av.loc[(slice(None),vehicle),'Battery_Use']
        for period in vehicle_prof.index.get_level_values(0):
            cumul_use = profile.loc[(slice(period),vehicle),'Battery_Use'].sum()
            cumul_profile = profile_av.loc[(slice(period),vehicle),'Battery_Use']
            prob += lpSum( # Doesn't go over 100% SOC
                [outputs[period, vehicle] * gv.CHARGER_EFF for period, vehicle in cumul_profile.index]
            ) + cumul_use + rel_charge[vehicle]<= 0
            prob += lpSum( # Doesn't go below 0% SOC
                [outputs[period, vehicle] * gv.CHARGER_EFF for period, vehicle in cumul_profile.index]
            ) + cumul_use + rel_charge[vehicle]>= -gv.BATTERY_CAPACITY + charger * gv.TIME_FRACT

    # Max capacity constraint
    n = len(time_periods.unique())
    for period in time_periods:
        time_routes = list(profile_av.loc[period].index)
        prob += lpSum(
            [outputs[period, route] for route in time_routes]) <= capacity[ca]* gv.TIME_FRACT

    # Solve and print to the screen
    prob.solve()
    print(ca, "Partial charge status:", LpStatus[prob.status])
    if prob.status ==-1:
        print('Magic!!')
        df = magic_charging(profile,ca,rel_charge)
    else:
        # Get output variables
        charge_output = []
        for period, route in outputs:
            if prob.status == 1:
                x =  outputs[(period, route)].varValue
            else:
                x = 0
            var_output = {
                'from': period,
                'Vehicle_ID': route,
                output_col: x
            }
            charge_output.append(var_output)

        df = pd.DataFrame.from_records(charge_output).sort_values(['from','Vehicle_ID'])
        df.set_index(['from', 'Vehicle_ID'], inplace=True)
        print('Cost:', value(prob.objective))
    return df

## Create a magic charging schedule to get back to 100% when all else has failed

def magic_charging(profile,ca,rel_charge):
    profile_av = profile[profile['Available'] == 1]
    #price_col = gv.CAT_COLS['PRICE'][ca]
    output_col = gv.CAT_COLS['OUTPUT'][ca]
    vehicles = profile.index.get_level_values(1).unique()
    time_periods = profile_av.index.get_level_values(0).unique()
    required_energy = profile['Battery_Use'].groupby('Vehicle_ID').sum() + rel_charge
    num_timeperiods = profile_av['Available'].groupby('Vehicle_ID').count()
    req_output = -required_energy/( gv.CHARGER_EFF* num_timeperiods)
    profile_av[output_col] = 0
    for idx in profile_av.index:
        profile_av.loc[idx,output_col] = req_output.loc[idx[1]]
    return profile_av[[output_col]]

In [ ]:
day = dt.datetime(2019,3,8)
day_profile = f.create_daily_schedule(empty_profile, day)

initial_rel_charge = pd.Series(
        data = [0,0,0,-20,0],
        index = empty_profile.index.get_level_values(1).unique()
    )

output_df, PuLP_prob, fsoc = linear_optimiser_V3(
                    day_profile,
                    ca,
                    charger,
                    site_capacity,
                    initial_rel_charge
                    )

day_profile_out = day_profile.merge(
                output_df,
                how='left',
                left_index=True,
                right_index=True,
                )
range_profile = day_profile_out.fillna(0)
cols=gv.CAT_COLS
vehicles = output_df.index.get_level_values(1).unique()

range_profile[cols['CHARGE_DEL'][ca]] = (
    range_profile[cols['OUTPUT'][ca]] 
    * gv.CHARGER_EFF)
range_profile[cols['ECOST'][ca]] = (
    range_profile[cols['OUTPUT'][ca]] 
    * range_profile[cols['PRICE']['opt']])
for vehicle in vehicles:
    range_profile.loc[(slice(None),vehicle),cols['SOC'][ca]] = (
        gv.BATTERY_CAPACITY + initial_rel_charge
        + range_profile.loc[(slice(None),vehicle),cols['CHARGE_DEL'][ca]].cumsum() 
        + range_profile.loc[(slice(None),vehicle),'Battery_Use'].cumsum()
        )*100/gv.BATTERY_CAPACITY

In [ ]:
fig, axs = plt.subplots(
    4,
    figsize=(12,10),
    sharex=True, 
    gridspec_kw={'hspace':0.1})

x = range_profile.unstack().index.strftime('%H:%M')
cols = gv.CAT_COLS

axs[0].plot(
    x, 
    range_profile.unstack()[cols['OUTPUT'][ca]]*2)
axs[0].legend(list(vehicles))
axs[0].set_ylabel('Output (kWh)')

axs[1].plot(
    x, 
    range_profile.unstack()[cols['ECOST'][ca]]/100)
axs[1].set_ylabel('Electricity Costs (GBP)')

axs[2].plot(
    x, 
    range_profile.unstack()[cols['SOC'][ca]])
axs[2].set_ylabel('SOC(%)')

axs[3].plot(
    x, 
    range_profile.unstack()[cols['PRICE']['opt']], 
    color=gv.FPS_PURPLE)
axs[3].set_ylabel('Electricity Price (p/kWh)')
for ax in fig.get_axes():
    ax.xaxis.set_major_locator(plt.MaxNLocator(24))
fig.show()